1) Create a generator for prime numbers:
Write a generator function that generates prime numbers. Each call to the generator should yield the next prime number.

In [2]:
def prime(n):
    for num in range(2, n + 1):
        if all(num % i != 0 for i in range(2, int(num ** 0.5) + 1)):
            yield num

In [4]:
prime_counter = prime(100)
print(next(prime_counter))
print(next(prime_counter))
print(next(prime_counter))

2
3
5


2) Create a generator to generate random numbers within a range:
Write a generator function that generates random numbers within a specified range. Each call to the generator should yield a random number.

In [5]:
def random_numbers(start, end):
    import random
    while True:
        yield random.randint(start, end)

In [8]:
random_generator = random_numbers(1, 20)
print(next(random_generator))
print(next(random_generator))
print(next(random_generator))

14
9
2


3) Create a generator to generate permutations of a list:
Write a generator function that generates all possible permutations of a given list. Each call to the generator should yield a different permutation.

In [1]:
def list_permutations(lists):
    from itertools import permutations
    for perm in permutations(lists):
        yield perm

In [3]:
example = [x for x in range(0, 3)]
permutation_generator = list_permutations(example)

for perm in permutation_generator:
    print(perm)

(0, 1, 2)
(0, 2, 1)
(1, 0, 2)
(1, 2, 0)
(2, 0, 1)
(2, 1, 0)


4) Implement a memoization decorator:
Write a decorator that caches the result of a function for given input arguments. 
Apply this decorator to a computationally expensive function and observe the improved performance by reusing cached results.

In [1]:
def memoize(func):
    cache = {}

    def wrapper(*args, **kwargs):
        key = (args, tuple(sorted(kwargs.items())))
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]

    return wrapper

In [3]:
#computationally expensive function
@memoize
def up_to_squared(n):
    n_list = range(0, n)
    squared = []
    for num in n_list:
        result = num ** 2
        squared.append(result)
    return squared

In [4]:
import time

start1 = time.time()
result = up_to_squared(10)
end1 = time.time()
execution1 = end1 - start1

start2 = time.time()
result = up_to_squared(10)
end2 = time.time()
execution2 = end2 - start2

print(execution1, execution2)


7.700920104980469e-05 5.340576171875e-05


5) Implement a retry decorator:
Write a decorator that retries the execution of a function a specified number of times in case of failures or exceptions. Apply this decorator to functions
that interact with external services to handle temporary failures gracefully.

In [22]:
import time
def replay(attempts = 5, delay = 1):
    def decorator(func):
        def wrapper(*args, **kwargs):
            retry = 0
            while retry < attempts:
                try:
                    return func(*args, **kwargs)
                except:
                    print("There is an error. Trying again...")
                    retry += 1
                    if retry < attempts:
                        time.sleep(delay)
                else:
                    break
            raise RuntimeError(f"Function {func.__name__} failed after {attempts} attempts.")
        return wrapper
    return decorator


In [23]:
import requests

@replay(attempts=6, delay = 2)
def connect_to_webpage(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text


In [24]:
# it's difficult to find temporary failures in internet
#it worked on the first try
url = "https://lt.wikipedia.org/wiki/Dirižablis"

content = connect_to_webpage(url)
print(content)


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" lang="lt" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Dirižablis – Vikipedija</title>
<script>document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";(function(){var cookie=document.cookie.match(/(?:^|; )ltwikimwclientprefs=(

In [35]:
# approach with random, creating a failure situation

import random

@replay(attempts=5, delay=2)
def connect_to_fake_service():
    if random.random() < 0.8:
        raise ConnectionError()
    else:
        return "Connected successfully."

result = connect_to_fake_service()
print(result)

There is an error. Trying again...
Connected successfully.


6) Create a rate-limiting decorator:
Write a decorator that limits the rate at which a function can be called. Apply this decorator to functions that should not be invoked
more than a certain number of times per second or minute.

In [39]:
import time

def rate_limit(max_calls = 2, per = 1):
    def decorator(func):
        times = []
        def wrapper(*args, **kwargs):
            current_time = time.time()
            times[:] = [t for t in times if t >= current_time - per]
            if len(times) >= max_calls:
                wait = times[0] + per - current_time
                time.sleep(wait)
                print("Slow down, too many function calls")
            result = func(*args, **kwargs)
            times.append(time.time())    
            return result
        return wrapper
    return decorator

In [40]:
@rate_limit(max_calls=1, per=3)
def some_function(n):
    return "aaa" * n

In [41]:
for i in range(10):
    some_function(i)

Slow down, too many function calls
Slow down, too many function calls
Slow down, too many function calls
Slow down, too many function calls
Slow down, too many function calls
Slow down, too many function calls
Slow down, too many function calls
Slow down, too many function calls
Slow down, too many function calls


7) Pivot a DataFrame based on column values:
Take a DataFrame with columns representing categories and values, and pivot it to reshape the data, with the category values as columns and corresponding values in the cells.

In [48]:
import pandas as pd

data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'Alice', 'Bob', 'Charlie'],
    'Category': ['A', 'B', 'A', 'B', 'A', 'B'],
    'Value1': [10, 20, 15, 25, 30, 35],
    'Value2': [5, 10, 8, 12, 15, 20]
}
df = pd.DataFrame(data)

pivot_df = df.pivot(index = "Name", columns = "Category", values = ["Value1", "Value2"])
pivot_df

Value1     Value2    
Category      A   B      A   B
Name                          
Alice        10  25      5  12
Bob          30  20     15  10
Charlie      15  35      8  20

8) Aggregate and pivot data:
Given a DataFrame with multiple columns, apply aggregation functions (e.g., sum, average) to the values and pivot the data based on specific columns.

In [51]:
pivot_df2 = df.pivot_table(index='Name', columns='Category', values=['Value1', 'Value2'], aggfunc=['sum', 'mean'])
pivot_df2

sum                  mean               
         Value1     Value2     Value1     Value2    
Category      A   B      A   B      A   B      A   B
Name                                                
Alice        10  25      5  12     10  25      5  12
Bob          30  20     15  10     30  20     15  10
Charlie      15  35      8  20     15  35      8  20

9) Pivot with multi-index columns:
Perform a pivot operation on a DataFrame with multi-index columns, reshaping the data based on specific levels of the column index.

In [9]:
import pandas as pd
import random 

students_subjects = [["Laura", "Monika", "Ugne", "Kamila"], ["Maths", "IT", "English"]]
grades = [[random.randint(4, 10), random.randint(4, 10)] for _ in range(12)]

In [10]:
index = pd.MultiIndex.from_product(students_subjects, names= ["Students", "Subjects"])
student_df = pd.DataFrame(grades, index=index, columns=["1st Semester", "2nd Semester"])
student_df

1st Semester  2nd Semester
Students Subjects                            
Laura    Maths               10             8
         IT                   9             8
         English             10            10
Monika   Maths               10             8
         IT                   9             4
         English             10             4
Ugne     Maths                4             9
         IT                   9             5
         English              5             7
Kamila   Maths                8             4
         IT                   4             8
         English              9             8

In [14]:
unstacked1 = student_df.unstack(level =1)
unstacked1

1st Semester          2nd Semester         
Subjects      English IT Maths      English IT Maths
Students                                            
Kamila              9  4     8            8  8     4
Laura              10  9    10           10  8     8
Monika             10  9    10            4  4     8
Ugne                5  9     4            7  5     9

In [17]:
unstacked0 = student_df.unstack(level =0)
unstacked0

1st Semester                   2nd Semester                  
Students       Kamila Laura Monika Ugne       Kamila Laura Monika Ugne
Subjects                                                              
English             9    10     10    5            8    10      4    7
IT                  4     9      9    9            8     8      4    5
Maths               8    10     10    4            4     8      8    9

10) Take any data (your choice) and visualize the data using Ploty library. You can select any plot type,
but you need to comment (write a comment in code) why the selected type of plot represents the data best

In [18]:
import pandas as pd

diabetes = pd.read_csv("diabetes.csv")
diabetes

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [33]:
# I chose scatter plot with no dot connection because
# every row represents different person and
# the purpose of this plot is to see the relation between 
# blood pressure and glucose. We can see that most of the patients
# have glucose range from ~70 to ~150 and blood pressure from
# ~60 to ~95 just looking at the plot visually.
# This graph for relation is one of the best because we can see
# the data of every patient and see potential outliers.
# Most of the other graphs would require categorizing or 
# other processing of this particular data

import plotly.graph_objects as go

fig = go.Figure(data = go.Scattergl(x = diabetes.Glucose, y = diabetes.BloodPressure, mode= 'markers'))
fig.update_layout(title= "Relation of Blood Pressure based on Glucose Concentration in Blood", xaxis_title = "Glucose concentration in Blood", yaxis_title = "Blood Pressure", showlegend = False)
fig.show()